In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from collections import defaultdict

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from collections import defaultdict
import numpy as np
import pandas as pd

def evaluate_models(csv_path):
    # Load CSV
    df = pd.read_csv(csv_path)

    # Drop unwanted columns
    df = df.drop(columns=["Unnamed: 0", "PID"], errors="ignore")

    # Extract identifier
    targets = ['arousal_category', 'valence_category']
    df = df.dropna(subset=targets + ['<dataset name colum>'])  # Remove rows with missing labels
    feature_cols = df.drop(columns=targets + ['<dataset name colum>']).columns

    # Initialize models
    models = {
        'MLP': MLPClassifier(random_state=42, max_iter=100),
        'RandomForest': RandomForestClassifier(random_state=42),
    }

    # Store metrics
    results = {target: defaultdict(list) for target in targets}

    # Leave-one-dataset-out cross-validation
    unique_ids = df['<dataset name colum>'].unique()

    for uid in unique_ids:
        test_idx = df['<dataset name colum>'] == uid
        train_idx = ~test_idx

        X_train = df.loc[train_idx, feature_cols]
        X_test = df.loc[test_idx, feature_cols]

        if X_train.empty or X_test.empty:
            print(f"Skipping <dataset name colum> {uid} due to empty train/test split.")
            continue

        X_train_scaled = X_train  # You may want to scale later
        X_test_scaled = X_test

        for target in targets:
            y_train = df.loc[train_idx, target]
            y_test = df.loc[test_idx, target]

            # Skip if training or test labels are missing or insufficient
            if y_train.nunique() < 2:
                print(f"Skipping target {target} for <dataset name colum> {uid} due to insufficient class diversity.")
                continue

            for model_name, model in models.items():
                try:
                    model.fit(X_train_scaled, y_train)
                    y_pred = model.predict(X_test_scaled)

                    acc = accuracy_score(y_test, y_pred)
                    f1 = f1_score(y_test, y_pred, average='weighted')

                    results[target][model_name + '_accuracy'].append(acc)
                    results[target][model_name + '_f1'].append(f1)
                except ValueError as e:
                    print(f"Model {model_name} failed on <dataset name colum> {uid} for target {target}: {e}")
                    continue

    # Report average metrics
    for target in targets:
        print(f"\nResults for target: {target}")
        for metric in results[target]:
            scores = results[target][metric]
            if scores:
                avg_score = np.mean(scores)
                print(f"{metric}: {avg_score:.4f}")
            else:
                print(f"{metric}: No valid results")


In [ ]:
print("Results for transferability within a group\n")
csv_path = "<path to csv for the group of datasets chosen for eda+ppg>"
print("Results for Combined")
evaluate_models(csv_path)
print()
csv_path = "<path to csv for the group of datasets chosen for eda>"
print("Results for EDA")
evaluate_models(csv_path)
print()
csv_path = "<path to csv for the group of datasets chosen for ppg>"
print("Results for PPG")
evaluate_models(csv_path)

In [ ]:
import sys
import os
# Get the absolute path to the "Scripts" folder
current_dir = os.getcwd()
scripts_dir = os.path.join(current_dir, "..", "Scripts")
sys.path.append(scripts_dir)
import clsp_com
import clsp_eda
import clsp_ppg

In [ ]:
print("Zero-shot on CLSP on <group name>")

print("<Group Name>")
eda_fet_path = "<path to csv for eda features for the group>"
clsp_eda.Arousal(eda_path= eda_fet_path)
clsp_eda.Valence(eda_path=eda_fet_path)
ppg_fet_path = "<path to csv for ppg features>"
clsp_ppg.Arousal(ppg_path= ppg_fet_path)
clsp_ppg.Valence(ppg_path=ppg_fet_path)
com_fet_path = "<path to csv for eda+ppg features for the group>"
clsp_com.Arousal(com_path= com_fet_path)
clsp_com.Valence(com_path=com_fet_path)
